# Purpose of this notebook

What you can do about phrases and patterns in spacy.


Before we get into it, it should be pointed out that
- you can express more complex types of patterns, see https://spacy.io/api/matcher#patterns
- You could extend this to more complex tasks, like maybe rule-based phrase and named entity extraction.
  - ...though you might base that on more specific existing code like PhraseMatcher and EntityRuler,
    which may work faster and/or annotate automatically.
  - and in the case of NER would probaly still be less effective than existing trained NER model components

We will demonstrate that somewhat in the process.

In [ ]:
import collections
import spacy.displacy
import spacy.matcher
from spacy.matcher import Matcher

import wetsuite.datasets
import wetsuite.helpers.spacy

In [143]:
# some example data to use later
some_cvdr_text        = wetsuite.datasets.load('cvdr-mostrecent-text').data.random_values(250)
some_rechtspraak_text = list( case.get('bodytext')   for case in wetsuite.datasets.load('rechtspraaknl-struc').data.random_values(250) )

[The things we can match tokens on](https://spacy.io/usage/rule-based-matching#adding-patterns-attributes) are mostly attributes already on the Token object, so if you've already worked with spacy, you should recognize most as things you can also access via code (e.g. from the [Token documentation](https://spacy.io/api/token#attributes) and our [spacy intro notebook](methods_nlp__spacy_basics.ipynb)):
- `ORTH` or `TEXT` - the text as-is
- `LENGTH` - length of TEXT
- `LOWER` - the lowercase version of the text
- `NORM` - the normalized version (seems to do things like resolve contractions, and otherwise often be the lemmatizer output?)
- `LEMMA` - lemmatixed
- `SHAPE` - alphabetic characters become X or x, numeric by d, and sequences of the same are truncated after 4, so e.g. Katherine80 would become Xxxxxdd
- `POS` - coarse tagging  (often following wider conventions), e.g. `NOUN`; many models follow [a relatively universal parts-of-speech set](https://universaldependencies.org/u/pos/)
  - (so often `ADJ`, `ADP`, `ADV`, `AUX`, `CCONJ`, `DET`, `INTJ`, `NOUN`, `NUM`, `PART`, `PRON`, `PROPN`, `PUNCT`, `SCONJ`, `SYM`, `VERB`, `X` -- but you should check for each model)
- `TAG` - finer tagging than POS  
  - (easily model/language specific, e.g. `'N|soort|ev|basis|zijd|stan'`)
- `MORPH` - morphological properties 
  - (easily model/language specific, something like `Gender=Com|Number=Sing`)
  - note that this supports `{"MORPH": {"IS_SUBSET": ["Number=Sing", "Gender=Neut"]}}`

- `DEP` - dependency relation in the parse ([this takes some more typing to specfy](https://spacy.io/usage/rule-based-matching#dependencymatcher))
- `IS_SENT_START`
- `IS_LOWER`, `IS_UPPER`, `IS_TITLE`; `IS_PUNCT`, `IS_SPACE`, `IS_STOP`; `IS_ALPHA`, `IS_ASCII`, `IS_DIGIT`; `LIKE_NUM`, `LIKE_URL`, `LIKE_EMAIL`
- `_` accesses custom attributes

Also,
* you can add the optional `OP`, a [quantifier](https://spacy.io/usage/rule-based-matching#quantifiers) (that uses a syntax muc like regexes)
  - `?`, `+`, `*`, `{n}`, `{n,m}`, `{n,}`, `{,m}`
  - ...and `!` which means it must _not_ match (match 0 times at this point) 
    - TOFIGURE: (does it consume so mean an "anything but this?" -- or does it _not_ consume and mean more of a negative lookahead?)
  - consider e.g. `{'POS':'ADJ', 'OP':'*'}, {"POS": {"REGEX": "(NOUN|PROPN)"}}`

* instead of the value being one literal scring, you can make it a dict and add one of the following:
  - use `==`, `>=`, `<=`, `>`, `<`
    - consider e.g. `{"LENGTH": {">=": 10}}`
  - use `IN`, `NOT_IN`, `IS_SUBSET`, `IS_SUPERSET`, `INTERSECTS` to compare with lists
    - consider e.g. `{"POS": {"IN":["NOUN","PROPN"]} }` or `{"MORPH": {"IS_SUBSET": ["Number=Sing", "Gender=Neut"]}}`

  - invoke [the regex operator](https://spacy.io/usage/rule-based-matching#regex)
    - consider e.g. `{"TEXT": {"REGEX": "deff?in[ia]tely"}}`, or `{"TAG": {"REGEX": "^N"}`
  - invoke [the fuzzy operator](https://spacy.io/usage/rule-based-matching#fuzzy)
    - which (by default) matches an [edit distance](https://en.wikipedia.org/wiki/Edit_distance#Example) of 2, so e.g. `{"FUZZY": "favorite"}` will match  "favourite", "favorites", and "gavorite".
    
  - some combinations, e.g. `{"FUZZY": {"IN": ["awesome", "cool", "wonderful"]}}}`

* 'match _any_ one token' can be done with `{}`
  - TOFIGURE: does that mean `{'OP':'+'}` matches one or more?

### How do put patterns in, how do I get matches out?

Both depends a little on whether you are are using a separate Matcher, or are adding a Ruler to the model object.

Chances are you'll end up on something more visual while experimenting,
and something more succinct once you're using this as a tool.

#### bare Matcher

A Matcher is a separate thing that you can run on a document. For example:

In [186]:
dutch  = spacy.load('nl_core_news_lg')

matcher = Matcher(dutch.vocab)
matcher.add("HW_MetPunct", [   [{"LOWER": "hallo"}, {"IS_PUNCT": True}, {"LOWER": "wereld"}]            ])
matcher.add("HW_Opt",      [   [{"LOWER": "hallo"}, {"IS_PUNCT": True, "OP":'*'}, {"LOWER": "wereld"}]  ])  # punctuation is optional in this one (so matches both in the example)

doc = dutch("De hallo, wereld test. Eerste patroon matcht niet op hallo wereld.")
print( list( repr(tok) for tok in doc ) ) # (just to point out what the tokens are)

for match_id, start, end in matcher( doc ):   # (end is exclusive)
    match_str  = dutch.vocab.strings[match_id] # (spacy makes a point of having an integer representation of everything it might store; this gets a get string representation of the pattern name, and is optional if you don't care)
    match_span = doc[start:end]  # fetch as span, mostly because it's the easiest way to get the .text again:
    print(f"Pattern {repr(match_str):15s} matches token {start:3d}..{end-1:<3d}  which is the text {repr(match_span.text)}")


['De', 'hallo', ',', 'wereld', 'test', '.', 'Eerste', 'patroon', 'matcht', 'niet', 'op', 'hallo', 'wereld', '.']
Pattern 'HW_MetPunct'   matches token   1..3    which is the text 'hallo, wereld'
Pattern 'HW_Opt'        matches token   1..3    which is the text 'hallo, wereld'
Pattern 'HW_Opt'        matches token  11..12   which is the text 'hallo wereld'


In [159]:
#As a slightly more interesting example, on some real data, let's look for  
#one or more adjectives,  before  <a noun or proper noun>.

an_pattern = [
    [ # you can have more rules in a matcher, this has just one 
        {"POS":"ADJ",   "OP":"+"},   
        {"POS":{"IN":["NOUN","PROPN"]} } 
    ],
]
matcher = spacy.matcher.Matcher(dutch.vocab)
matcher.add("adjective-noun", an_pattern)

doc = dutch( some_cvdr_text[0] )
for _, start, end in matcher(doc):
    print( doc[start:end].text, end=';   ' )

# 


bijzondere commissie;   derde lid;   bijzondere commissie;   eerste lid;   politieke ambtsdragers;   decentrale politieke ambtsdragers;   openbaar vervoer;   eigen auto;   eigen auto;   functionele beperking;   tijdelijke functionele beperking;   eerste lid;   geschikte vervoersvoorziening;   werkelijke verblijfkosten;   eerste lid;   politieke ambtsdragers;   decentrale politieke ambtsdragers;   eerste lid;   tijdelijk ontslag;   Nadere regels;   politieke ambtsdragers;   decentrale politieke ambtsdragers;   inhoudelijke informatie;   maximale vergoeding;   overlegde stukken;   bijzondere deskundigheid;   zwaarte taak;   eerste lid;   politieke ambtsdragers;   decentrale politieke ambtsdragers;   beroepsmatige deskundigheid;   bijzondere beroepsmatige deskundigheid;   redelijke verhouding;   politieke ambtsdragers;   decentrale politieke ambtsdragers;   eerste lid;   politieke ambtsdragers;   decentrale politieke ambtsdragers;   eerste lid;   tweede lid;   vaste vergoedingen;   politi

#### EntityRuler, or preferably SpanRuler

Sometimes we're using spacy to collect and comparing different things,
so can we have it marked up on the document?

Yes, either with a Spanruler (ends up in `.spans`) or an EntityRuler (ends up in `.ents`).
Both can also be visualized.

<!-- -->

Using an EntityRuler makes it easy to visualize in exactly, but has some problems:
Matches are silently dropped when it overlaps with an already marked up entity (you could disable the existing NER),
or with another of your own matches (there is no way to avoid that; this is awkward at best).

<!-- -->

Because spans can overlap, adding with a SpanRuler is complete.
Visualizing it isn't as compact, though, _because_ it is capable of displaying overlapping spans. 
Note that `.spans` is actually a dict from a useful name, to a list of spans, 
so that different things can put distinct things into that attrivute.
SpanRuler defaults to add in the key 'ruler', which is why we have to explicitly tell it that below.

Note that the way you hand in the pattern is a little different from the above, and the same between these two:

In [184]:
# (these fragments repeat code to be relatively self-contained)
dutch_with_er  = spacy.load('nl_core_news_lg')
#dutch.remove_pipe("ner") # happens not to clash here, so we can leave them in
ruler = dutch_with_er.add_pipe("entity_ruler")
ruler.add_patterns( [ {'label':'ADJ_N', "pattern":[ {"POS":"ADJ", "OP":"+"},  {"POS":{"IN":["NOUN","PROPN"]} }  ]} ] )

doc = dutch_with_er("""Onder de naam "afvalstoffenheffing" wordt een directe belasting geheven als bedoeld inartikel 15.33 van de Wet milieubeheer;artikel 15.33 van de Wet milieubeheer; de afvalstoffenheffing als bedoeld in deze verordening en de daarbij behorende tarieventabel wordt naar afzonderlijke grondslagen geheven ter zake van het feitelijk gebruik van een perceel ten aanzien waarvan krachtensartikel 10.21en10.22 van de Wet milieubeheereen verplichting tot het inzamelen van huishoudelijke afvalstoffen geldt.artikel 10.21en10.22 van de Wet milieubeheereen verplichting tot het inzamelen van huishoudelijke afvalstoffen geldt.""")
spacy.displacy.render(doc, style='ent', jupyter=True)

In [185]:
dutch_with_sr  = spacy.load('nl_core_news_lg')
ruler = dutch_with_sr.add_pipe("span_ruler")
ruler.add_patterns( [ {'label':'ADJ_N', "pattern":[  {"POS":"ADJ",  "OP":"+"},  {"POS":{"IN":["NOUN","PROPN"]} }  ],} ] )

doc = dutch_with_sr("""Onder de naam "afvalstoffenheffing" wordt een directe belasting geheven als bedoeld inartikel 15.33 van de Wet milieubeheer;artikel 15.33 van de Wet milieubeheer; de afvalstoffenheffing als bedoeld in deze verordening en de daarbij behorende tarieventabel wordt naar afzonderlijke grondslagen geheven ter zake van het feitelijk gebruik van een perceel ten aanzien waarvan krachtensartikel 10.21en10.22 van de Wet milieubeheereen verplichting tot het inzamelen van huishoudelijke afvalstoffen geldt.artikel 10.21en10.22 van de Wet milieubeheereen verplichting tot het inzamelen van huishoudelijke afvalstoffen geldt.""")
doc.spans

spacy.displacy.render(doc, style='span', options={"spans_key":"ruler"}, jupyter=True)

<!--
#### Manually

When you have specific things you want to do, you might get more creative with code.

Say, "I want only adjective-noun pairs from the sentences that mention appellant(/e/es), and want to count how of they they occur":

count = collections.defaultdict(int)
for text in some_rechtspraak_text:
    doc = dutch( text )
    for sent in doc.sents:
        if not 'appellant' in sent.text.lower():
            continue
        for _, start_i, end_i in matcher( sent ):
            count[ sent[ start_i : end_i ].text.strip() ] += 1 

for str, count in sorted( count.items(), key=lambda x:x[1], reverse=True):
    print( f'{count:5d}  {str}')
    #if count<3:
    #    break
-->

### Making life a little easier?

These rules are... verbose.  We can make a little rule system to make that less typing.

Note that this one is _less_ expressive.

In [209]:
import spacy
from spacy.matcher import Matcher
import wetsuite.datasets

def expand_pats(patstring):
    '  '
    pats = {
        'DPN':      [{'POS':'DET', 'OP':'*'}, {'POS':'PRON', 'OP':'*'}, {"POS": {"REGEX": "(NOUN|PROPN)"}}],
        'ADJN':     [{'POS':'DET', 'OP':'*'}, {'POS':'ADJ', 'OP':'*'}, {"POS": {"REGEX": "(NOUN|PROPN)"}}],
        'ADJ':      [{'POS':'ADJ', 'OP':'+'}],
        'V':        [{"POS": {"IN": ["VERB", "AUX"]}}],
        #'V':        [{"POS": {"REGEX": "(VERB|AUX)"}}],
        #'ANYPLUS':  [{"TEXT": {"REGEX": "."}, 'OP':'+'}], #figure out whether there is a better/faster way to match any token
        #'ANYSTAR':  [{"TEXT": {"REGEX": "."}, 'OP':'*'}],
        'ANYPLUS':  [{'OP':'+'}], #figure out whether there is a better/faster way to match any token
        'ANYSTAR':  [{'OP':'*'}],
        'DATELIKE': [{"LIKE_NUM": True}, {"POS": "PROPN"}, {"LIKE_NUM": True}],
    }
    ret = []
    for part in patstring.split():
        if part == part.lower():
            ret.append( {'LOWER':part} )
        else:
            if part in pats:
                ret.extend( pats[part] )
            else:
                print('Do not know pattern %r'%part)
    return ret

matcher = Matcher(dutch.vocab)
for pat in (
    #'ADJN heeft V dat',
    #'zou zijn V door',
    #'gelet op de ADJN',
    #'ADJ rechten',
    #'de beschuldigingen ANYPLUS zijn',
    #'aan de hand van het ADJN',
    #'aan de hand van de ADJN',
    #'volgens ADJN',
    #'gaat ADJN uit van',
    #'gaan ADJN uit van',
 #   'behandeld op DATELIKE',
 #   'uitgesproken ANYSTAR op DATELIKE',
 #   'uitgesproken ANYSTAR van DATELIKE',
 #   'uitspraak ANYPLUS DATELIKE',
    #'voorlopig karakter', 'niet bindend', #'bodemprocedure',
    #'nadere stukken',
    #'ADJN concludeert',
    #'ADJN voert verweer'
    ):
    #print(pat)
    ep = expand_pats(pat)
    #print(ep)
    matcher.add( pat, [ep] ) 

matcher.add( 'deze v|u V', [ [ 
        {"LOWER":   {"IN":["deze", "dit"]}},   
        {"LOWER":   {"IN":["verordening", "uitspraak", "arrest", "beschikking", "vonnis"]}},   
        {"POS": {"IN":["VERB","AUX"]} },
        {'OP':'+'},
        {"LIKE_NUM": True}, {"POS": "PROPN"}, {"LIKE_NUM": True}
], ])

In [210]:
for case in wetsuite.datasets.load('rechtspraaknl-struc').data.random_values(250):
    print(case['identifier'])

    doc = dutch( case.get('bodytext') )

    for sent in doc.sents:
        for match_id, start, end in matcher( sent ):
            match_str = dutch.vocab.strings[match_id]  # Get string representation, seems to point out the pattern name is added to the vocab too, presumably to have an integer-only representation?
            match_span = sent[start:end]  # The matched span
            print(f"  {repr(match_str):35s} matches:  {repr(match_span.text)}")    

ECLI:NL:RVS:2010:BN3203
  'deze v|u V'                        matches:  'deze uitspraak heeft [appellant] bij brief, bij de Raad van State ingekomen op 7 september 2009'
ECLI:NL:PHR:2021:1010
ECLI:NL:GHSHE:2022:4623
ECLI:NL:RBMID:2011:BR1104
  'deze v|u V'                        matches:  'Dit vonnis is gewezen door mr. M.J.M. Klarenbeek, kantonrechter, en uitgesproken ter openbare terechtzitting van 20 april 2011'
ECLI:NL:HR:2022:1419
  'deze v|u V'                        matches:  'Deze beschikking is gegeven door de vice-president V. van den Brink als voorzitter, en de raadsheren A.L.J. van Strien en M. Kuijer, in bijzijn van de waarnemend griffier H.J.S. Kea, en uitgesproken ter openbare terechtzitting van\n\n11 oktober 2022'
ECLI:NL:RBMNE:2023:5756
  'deze v|u V'                        matches:  'Dit vonnis is op tegenspraak gewezen naar aanleiding van het onderzoek op de terechtzitting van 20 oktober 2023'
ECLI:NL:RBAMS:2024:3479
ECLI:NL:RBUTR:2008:BG2578
ECLI:NL:RBDHA:2020:1478


KeyboardInterrupt: 